In [ ]:
pip install tensorflow

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
df=pd.read_csv("insurance_data.csv")
df.head()

In [ ]:
# splitting fromṁataset_sp

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=25)

In [ ]:
# split train and test

In [ ]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

model.fit(X_train_scaled,y_train,epochs=5000)

In [ ]:
model.evaluate(X_test_scaled,y_test)

In [ ]:
model.predict(X_test_scaled)

In [ ]:
y_test

In [ ]:
coef,intercept=model.get_weights()

In [ ]:
coef,intercept

In [ ]:
def signmoid(x):
  import math
  return 1 / (1+math.exp(-x))

signmoid(18)

In [ ]:
X_test

In [ ]:
def prediction_function(age,affordability):
  weighted_sum = coef[0]*age + coef[1]*affordability+intercept
  return signmoid(weighted_sum)

prediction_function(.47,1)

In [ ]:
def prediction_function(age,affordability):
  weighted_sum = coef[0]*age + coef[1]*affordability+intercept
  return signmoid(weighted_sum)
prediction_function(.18,1)

In [ ]:
# now we start implementing graient descent in python .again goal is to come up with same w1,w2 and bias that kkeras 

In [ ]:
# write 1st couple of helper routines such as sigmois and log_loss

In [ ]:
def sigmoid_numpy(X):
  return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

In [ ]:
def log_loss(y_true,y_predicted):
  epsilon =1e-15
  y_predicted_new = [max(i,epsilon) for i in y_predicted]
  y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [ ]:
def gradient_descent(age,affordability,y_true,epochs,loss_threshold):
  w1=w2=1
  bias=0
  rate = 0.5
  n = len(age)
  for i in range(epochs):
    weighted_sum = w1+w2*affordability + bias
    y_predicted = sigmoid_numpy(weighted_sum)
    loss = log_loss(y_true,y_predicted)

    w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
    w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))

    bias_d = np.mean(y_predicted-y_true)
    w1 = w1 - rate * w1d
    w2 = w2 - rate * w2d
    bias = bias -rate * bias_d

    print(f'Epoch:{i},w1:{w1},w2:{w2},bias:{bias},loss:{loss}')

    if loss<=loss_threshold:
      break
  return w1,w2,bias
  
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000,0.4631)

In [ ]:
coef,intercept